# Grupo 5

In [44]:
# Importando paquetes:
using Pkg
using CSV
using Distributions
using DataFrames
using Dates
using Plots
using Random
using LinearAlgebra
using LaTeXStrings
using Lasso
using Statistics
using GLMNet
using MLJ

# 1. Loading and processing the data 

1.1. Load the data, either from the url or from the path provided

In [45]:
data = CSV.read("wage2015_subsample_inference.csv", DataFrame)
select!(data, Not("rownames"));
first(data,5)

Row,rownames,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,10,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,12,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,15,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,18,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,19,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22


1.2. As in Group Assignment 1 2024 - 2 #1044, generate the extra-flexible model. This means that it contains all two-way interactions between the experience polynomials and the indicator variables

1.2.1. Generate the array for the outcome variable Y and normalize it

In [46]:
#Se selecciona a la variable dependiente que será "lwage" en este caso es más fácil normalizar la variable en logaritmos
#Esto mejora la asimetría que puede tener la variable de salarios
lwage = data.lwage
#Dado que se tiene el array ahora se normaliza con el siguiente código
Y = (lwage .- mean(lwage)) ./ std(lwage)

first(Y,5)

5-element Vector{Float64}:
 -1.2402545501637494
  1.5814159439013908
 -0.9952235701262604
 -0.588828412034688
  0.6858352078834455

1.2.2. Generate the array for the predictors X (do not generate an intercept) and normalize its colums.

In [47]:
# Número de valores a transformar en dummys
println(sort(unique(data.occ2)))
println(sort(unique(data.ind2)))

In [48]:
for i in 2:22
    col = Symbol("occ2_$i")
    transform!(data, :occ2 => ByRow(x -> x == i ? 1 : 0) => col)
end

for i in 2:21
    col = Symbol("ind2_$i")
    transform!(data, :ind2 => ByRow(x -> x == i ? 1 : 0) => col)
end

println(data)

Se crea la base de datos que serán las features del modelo "x"

In [49]:
v = [
    :exp1, :exp2, :exp3, :exp4,
    :hsg, :scl, :clg, :ad, :so, :we, :ne,   
    :occ2_2, :occ2_3, :occ2_4, :occ2_5, :occ2_6, :occ2_7, :occ2_8, :occ2_9, 
    :occ2_10, :occ2_11, :occ2_12, :occ2_13, :occ2_14, :occ2_15, :occ2_16, 
    :occ2_17, :occ2_18, :occ2_19, :occ2_20, :occ2_21, :occ2_22,
    :ind2_2, :ind2_3, :ind2_4, :ind2_5, :ind2_6, :ind2_7, :ind2_8, :ind2_9,
    :ind2_10, :ind2_11, :ind2_12, :ind2_13, :ind2_14, :ind2_15, :ind2_16,
    :ind2_17, :ind2_18, :ind2_19, :ind2_20, :ind2_21
]
# se crea un nuevo DataFrame para almacenar solo las interacciones
interactions = DataFrame()

# Añadir la variable "sex" del DataFrame original
interactions[!, :sex] = data[!, :sex]

# Se generan las interacciones de dos vías entre las variables en `v`
for i in 1:length(v)
    for j in (i+1):length(v) 
        var1 = v[i]
        var2 = v[j]

        # Nombre de la nueva interacción
        interac2 = Symbol("$(var1)_x_$(var2)")

        # producto entre las variables
        interactions[!, interac2] = data[!, var1] .* data[!, var2]
    end
end

# Mostrar las primeras filas del DataFrame con solo las interacciones y "sex"
first(interactions, 5)

Row,sex,exp1_x_exp2,exp1_x_exp3,exp1_x_exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,exp1_x_ind2_16,exp1_x_ind2_17,exp1_x_ind2_18,exp1_x_ind2_19,exp1_x_ind2_20,exp1_x_ind2_21,exp2_x_exp3,exp2_x_exp4,exp2_x_hsg,exp2_x_scl,exp2_x_clg,exp2_x_ad,exp2_x_so,exp2_x_we,exp2_x_ne,exp2_x_occ2_2,exp2_x_occ2_3,exp2_x_occ2_4,exp2_x_occ2_5,exp2_x_occ2_6,exp2_x_occ2_7,exp2_x_occ2_8,exp2_x_occ2_9,exp2_x_occ2_10,exp2_x_occ2_11,exp2_x_occ2_12,exp2_x_occ2_13,exp2_x_occ2_14,exp2_x_occ2_15,exp2_x_occ2_16,exp2_x_occ2_17,exp2_x_occ2_18,exp2_x_occ2_19,exp2_x_occ2_20,exp2_x_occ2_21,exp2_x_occ2_22,exp2_x_ind2_2,exp2_x_ind2_3,exp2_x_ind2_4,exp2_x_ind2_5,exp2_x_ind2_6,exp2_x_ind2_7,exp2_x_ind2_8,exp2_x_ind2_9,exp2_x_ind2_10,exp2_x_ind2_11,exp2_x_ind2_12,exp2_x_ind2_13,exp2_x_ind2_14,exp2_x_ind2_15,exp2_x_ind2_16,exp2_x_ind2_17,exp2_x_ind2_18,exp2_x_ind2_19,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1.0,3.43,2.401,1.6807,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.16807,0.117649,0.0,0.0,0.49,0.0,0.0,0.0,0.49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.49,0.0,⋯
2,0.0,297.91,923.521,2862.92,0.0,0.0,31.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,286.292,887.504,0.0,0.0,9.61,0.0,0.0,0.0,9.61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,0.0,58.32,104.976,188.957,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.8957,34.0122,3.24,0.0,0.0,0.0,0.0,0.0,3.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.24,0.0,0.0,0.0,0.0,0.0,3.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,1.0,156.25,390.625,976.562,0.0,0.0,0.0,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.6562,244.141,0.0,0.0,0.0,6.25,0.0,0.0,6.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
5,1.0,106.48,234.256,515.363,0.0,0.0,22.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,

Se crean el array para las features y se normaliza

In [50]:
X = Matrix(select(interactions, Not(:sex)))

# Normalizando las columnas de X sin sex porque es categórica
X_normalized = (X .- mean(X, dims=1)) ./ std(X, dims=1)

# se crea un nuevo DataFrame con las variables normalizadas
df_normalized = DataFrame(X_normalized, names(select(interactions, Not(:sex))))

# Se añade la columna `sex` al principio del DataFrame para coincidir con la estructura del modelo
insertcols!(df_normalized, 1, :sex => interactions[!, :sex])

first(df_normalized, 5)

Row,sex,exp1_x_exp2,exp1_x_exp3,exp1_x_exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,exp1_x_ind2_16,exp1_x_ind2_17,exp1_x_ind2_18,exp1_x_ind2_19,exp1_x_ind2_20,exp1_x_ind2_21,exp2_x_exp3,exp2_x_exp4,exp2_x_hsg,exp2_x_scl,exp2_x_clg,exp2_x_ad,exp2_x_so,exp2_x_we,exp2_x_ne,exp2_x_occ2_2,exp2_x_occ2_3,exp2_x_occ2_4,exp2_x_occ2_5,exp2_x_occ2_6,exp2_x_occ2_7,exp2_x_occ2_8,exp2_x_occ2_9,exp2_x_occ2_10,exp2_x_occ2_11,exp2_x_occ2_12,exp2_x_occ2_13,exp2_x_occ2_14,exp2_x_occ2_15,exp2_x_occ2_16,exp2_x_occ2_17,exp2_x_occ2_18,exp2_x_occ2_19,exp2_x_occ2_20,exp2_x_occ2_21,exp2_x_occ2_22,exp2_x_ind2_2,exp2_x_ind2_3,exp2_x_ind2_4,exp2_x_ind2_5,exp2_x_ind2_6,exp2_x_ind2_7,exp2_x_ind2_8,exp2_x_ind2_9,exp2_x_ind2_10,exp2_x_ind2_11,exp2_x_ind2_12,exp2_x_ind2_13,exp2_x_ind2_14,exp2_x_ind2_15,exp2_x_ind2_16,exp2_x_ind2_17,exp2_x_ind2_18,exp2_x_ind2_19,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1.0,-0.54475,-0.464746,-0.402714,-0.464634,-0.479743,0.460768,-0.282052,-0.469926,-0.399895,0.459553,-0.189084,-0.164887,-0.105012,-0.0843128,-0.117353,-0.0992162,-0.170651,-0.130965,-0.186553,3.05994,-0.112283,-0.17089,-0.128506,-0.117952,-0.267504,-0.305947,-0.0256204,-0.181304,-0.168245,-0.218414,-0.183369,-0.0750705,-0.0879398,-0.206714,-0.161349,-0.228863,-0.097307,-0.0888248,-0.288998,-0.138714,-0.149072,-0.183077,-0.102547,-0.232832,-0.0401832,-0.152511,-0.215209,0.907756,-0.110182,-0.213949,-0.170167,-0.402714,-0.354353,-0.3499,-0.348941,-0.096376,-0.214932,-0.332615,-0.292646,-0.110399,-0.140779,-0.122406,-0.0744491,-0.0598365,-0.0876263,-0.0718065,-0.127619,-0.10275,-0.142227,0.70192,-0.0841077,-0.121438,-0.10636,-0.0859652,-0.19588,-0.233915,-0.0214245,-0.136872,-0.129669,-0.169675,-0.147167,-0.0568023,-0.0743384,-0.156489,-0.130108,-0.177426,-0.0760114,-0.0710597,-0.212609,-0.10969,-0.118396,-0.135389,-0.0761664,-0.163898,-0.0327482,-0.114751,-0.164938,0.0738164,-0.0827462,⋯
2,0.0,1.48769,1.256,1.00973,-0.464634,-0.479743,3.60566,-0.282052,-0.469926,-0.399895,3.42634,-0.189084,-0.164887,-0.105012,-0.0843128,-0.117353,-0.0992162,-0.170651,-0.130965,7.702,-0.1065,-0.112283,-0.17089,-0.128506,-0.117952,-0.267504,-0.305947,-0.0256204,-0.181304,-0.168245,-0.218414,-0.183369,-0.0750705,-0.0879398,-0.206714,-0.161349,-0.228863,-0.097307,-0.0888248,4.95223,-0.138714,-0.149072,-0.183077,-0.102547,-0.232832,-0.0401832,-0.152511,-0.215209,-0.294934,-0.110182,-0.213949,-0.170167,1.00973,0.778379,-0.3499,-0.348941,4.01792,-0.214932,-0.332615,-0.292646,3.59087,-0.140779,-0.122406,-0.0744491,-0.0598365,-0.0876263,-0.0718065,-0.127619,-0.10275,8.41241,-0.0787276,-0.0841077,-0.121438,-0.10636,-0.0859652,-0.19588,-0.233915,-0.0214245,-0.136872,-0.129669,-0.169675,-0.147167,-0.0568023,-0

Finalmente, se crea el modelo fexible

In [51]:
# Se agrega 'df_normalized' en un DataFrame que incluya la target
model = hcat(DataFrame(Y = Y), df_normalized)
rename!(model, :Y => :wage_norm)
first(model, 5)


Row,wage_norm,sex,exp1_x_exp2,exp1_x_exp3,exp1_x_exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,exp1_x_ind2_16,exp1_x_ind2_17,exp1_x_ind2_18,exp1_x_ind2_19,exp1_x_ind2_20,exp1_x_ind2_21,exp2_x_exp3,exp2_x_exp4,exp2_x_hsg,exp2_x_scl,exp2_x_clg,exp2_x_ad,exp2_x_so,exp2_x_we,exp2_x_ne,exp2_x_occ2_2,exp2_x_occ2_3,exp2_x_occ2_4,exp2_x_occ2_5,exp2_x_occ2_6,exp2_x_occ2_7,exp2_x_occ2_8,exp2_x_occ2_9,exp2_x_occ2_10,exp2_x_occ2_11,exp2_x_occ2_12,exp2_x_occ2_13,exp2_x_occ2_14,exp2_x_occ2_15,exp2_x_occ2_16,exp2_x_occ2_17,exp2_x_occ2_18,exp2_x_occ2_19,exp2_x_occ2_20,exp2_x_occ2_21,exp2_x_occ2_22,exp2_x_ind2_2,exp2_x_ind2_3,exp2_x_ind2_4,exp2_x_ind2_5,exp2_x_ind2_6,exp2_x_ind2_7,exp2_x_ind2_8,exp2_x_ind2_9,exp2_x_ind2_10,exp2_x_ind2_11,exp2_x_ind2_12,exp2_x_ind2_13,exp2_x_ind2_14,exp2_x_ind2_15,exp2_x_ind2_16,exp2_x_ind2_17,exp2_x_ind2_18,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,-1.24025,1.0,-0.54475,-0.464746,-0.402714,-0.464634,-0.479743,0.460768,-0.282052,-0.469926,-0.399895,0.459553,-0.189084,-0.164887,-0.105012,-0.0843128,-0.117353,-0.0992162,-0.170651,-0.130965,-0.186553,3.05994,-0.112283,-0.17089,-0.128506,-0.117952,-0.267504,-0.305947,-0.0256204,-0.181304,-0.168245,-0.218414,-0.183369,-0.0750705,-0.0879398,-0.206714,-0.161349,-0.228863,-0.097307,-0.0888248,-0.288998,-0.138714,-0.149072,-0.183077,-0.102547,-0.232832,-0.0401832,-0.152511,-0.215209,0.907756,-0.110182,-0.213949,-0.170167,-0.402714,-0.354353,-0.3499,-0.348941,-0.096376,-0.214932,-0.332615,-0.292646,-0.110399,-0.140779,-0.122406,-0.0744491,-0.0598365,-0.0876263,-0.0718065,-0.127619,-0.10275,-0.142227,0.70192,-0.0841077,-0.121438,-0.10636,-0.0859652,-0.19588,-0.233915,-0.0214245,-0.136872,-0.129669,-0.169675,-0.147167,-0.0568023,-0.0743384,-0.156489,-0.130108,-0.177426,-0.0760114,-0.0710597,-0.212609,-0.10969,-0.118396,-0.135389,-0.0761664,-0.163898,-0.0327482,-0.114751,-0.164938,0.0738164,⋯
2,1.58142,0.0,1.48769,1.256,1.00973,-0.464634,-0.479743,3.60566,-0.282052,-0.469926,-0.399895,3.42634,-0.189084,-0.164887,-0.105012,-0.0843128,-0.117353,-0.0992162,-0.170651,-0.130965,7.702,-0.1065,-0.112283,-0.17089,-0.128506,-0.117952,-0.267504,-0.305947,-0.0256204,-0.181304,-0.168245,-0.218414,-0.183369,-0.0750705,-0.0879398,-0.206714,-0.161349,-0.228863,-0.097307,-0.0888248,4.95223,-0.138714,-0.149072,-0.183077,-0.102547,-0.232832,-0.0401832,-0.152511,-0.215209,-0.294934,-0.110182,-0.213949,-0.170167,1.00973,0.778379,-0.3499,-0.348941,4.01792,-0.214932,-0.332615,-0.292646,3.59087,-0.140779,-0.122406,-0.0744491,-0.0598365,-0.0876263,-0.0718065,-0.127619,-0.10275,8.41241,-0.0787276,-0.0841077,-0.121438,-0.10636,-0.0859652,-0.19588,-0.233915,-0.0214245,-0.136872,-0.129669,-0.169675,-0.147167,-0.0568023,-

1.3. Split between training and testing samples. The testing sample should be 10% of the total.

In [52]:
# Semilla 
Random.seed!(611)
# Tamaño del conjunto de prueba (10% del total)
n = size(model, 1)  # Número total de observaciones
test_size = Int(round(0.1 * n))  # 10% de los datos

# índices aleatorios para dividir en entrenamiento y prueba
indices = shuffle(1:n)  # Barajar todos los índices
train_indices = indices[1:end-test_size]  # 90% para entrenamiento
test_indices = indices[end-test_size+1:end]  # 10% para prueba

# Se tiene por separado a las features (excluyendo la target 'wage_norm') y la target
features = select(model, Not(:wage_norm))  # DataFrame de las features (excluyendo 'wage_norm')
target = model[!, :wage_norm]  # Array de la variable target 'wage_norm'

# se crean los conjuntos de entrenamiento y prueba por separado
# Conjuntos de entrenamiento
X_train = features[train_indices, :]
Y_train = target[train_indices]

# Conjuntos de prueba
X_test = features[test_indices, :]
Y_test = target[test_indices]

# Se muestran las dimensiones de los conjuntos
println("Tamaño de X_train: ", size(X_train))
println("Tamaño de Y_train: ", length(Y_train))
println("Tamaño de X_test: ", size(X_test))
println("Tamaño de Y_test: ", length(Y_test))

In [66]:
@show size(X_test)

(515, 1327)

# 2. Creating the Lasso Cross-Validation Procedure

2.4. Program a function that generates a logarithmically spaced grid. The input arguments should be the lower and upper bounds of the grid, as well as the natural logarithm of the spacing between each element of the grid. The output should be the logarithmically spaced grid, meaning that if we take the natural logarithm of each entry in the grid, they will be equally spaced. This will be the grid of values for λ values to try during cross-validation.

In [54]:
# Función logarítmicamente espaciada
function log_grid(lower_bound::Float64, upper_bound::Float64, log_spacing::Float64)
    # El número de puntos en la cuadrícula depende de la diferencia de los logaritmos
    num_points = Int(round((log(upper_bound) - log(lower_bound)) / log_spacing)) + 1
    
    # Generar los puntos espaciados uniformemente en el espacio logarítmico
    log_grid = range(log(lower_bound), stop=log(upper_bound), length=num_points)
    
    # Volver a la escala original exponenciando los valores
    return exp.(log_grid)
end

# Ejemplo (no sé a qué cuadrícula se refiere jeje)
lower_bound = 0.001  # Limite inferior más pequeño
upper_bound = 1.0   # Limite superior
log_spacing = 0.1   # Espaciado en la escala logarítmica  # Espaciado entre los puntos en la escala logarítmica

# Cuadrícula logarítmica
λ_grid = log_grid(lower_bound, upper_bound, log_spacing)

# Resultado
first(λ_grid,5)

5-element Vector{Float64}:
 0.0010000000000000002
 0.001105295141126022
 0.0012216773489967926
 0.001350314037869874
 0.0014924955450518293

2.5. Program a function to generate k folds. It should take as input the array to be split rowwise and the number of folds desired. It should output a list of k 1d arrays of booleans; these arrays should all be the same length as the number of rows in the input array, and when they are all summed together they should add up to an array of all true values. Create your own procedure for splitting. You can aid yourself with third party packages like numpy in Python or Statistics in Julia, but do not use a pre-programmed third party splitting procedure like sk-learns's KFolds in Python.

In [55]:
# Función adaptada para generar k pliegues (adaptada para cualquier número de filas)
function generate_k_folds(data::AbstractMatrix, k::Int)
    n = size(data, 1)  # Número de filas en la matriz de entrada
    fold_size = div(n, k)  # Tamaño base de cada pliegue
    remainder = n % k      # Filas adicionales a distribuir entre los primeros pliegues
    
    # Lista de booleanos para cada pliegue
    folds = fill(false, n, k)  # Crear una matriz de booleanos (n x k), inicializada en false
    
    # Crear una permutación aleatoria de los índices
    indices = shuffle(1:n)
    
    # Dividir los índices en k pliegues
    start_idx = 1
    for i in 1:k
        # Ajustar el tamaño del pliegue para distribuir las filas adicionales
        current_fold_size = fold_size + (i <= remainder ? 1 : 0)
        end_idx = start_idx + current_fold_size - 1
        
        # Asignar true a los pliegues correspondientes
        for idx in indices[start_idx:end_idx]
            folds[idx, i] = true
        end
        
        start_idx = end_idx + 1
    end
    
    # Convertir cada columna en un vector unidimensional de booleanos
    fold_vectors = [folds[:, i] for i in 1:k]
    
    return fold_vectors
end

# Ejemplo de uso
data = rand(515, 1327)  # Supongamos que tienes 500 filas, 1327 columnas (features)
k = 5  # Número de pliegues

# Generar los k pliegues
folds = generate_k_folds(data, k)

# Mostrar el resultado
for i in 1:k
    println("Pliegue $i: ", folds[i])
end

2.6. Program a function that integrates those that you programmed in the last two items to find the value of λ that minimizes the testing mean square error across folds. It should take the following inputs: 
...
With this, you can get the testing MSE associated with the value of lambda in each iteration. Feel free to use a third party Lasso estimator, as long as it is not one that calculates the optimal lambda value. Instead, the third party estimator you use should only estimate the regression with the penalty weight provided so you can get the testing MSE with that specific weight.

In [56]:
# Función para realizar la validación cruzada y encontrar el lambda óptimo
function cross_validation_lasso(Y_train, X_train, all_lambdas, k_folds)
    # Crear la matriz para almacenar los MSEs (dimensiones: lambdas x pliegues)
    all_mse = zeros(length(all_lambdas), length(k_folds))
    
    # Evaluar cada lambda en los pliegues
    for (lambda_idx, lambda_val) in enumerate(all_lambdas)
        for (fold_idx, test_mask) in enumerate(k_folds)
            # Convertir el vector booleano en índices
            train_indices = findall(.!test_mask)  # Índices para el conjunto de entrenamiento
            test_indices = findall(test_mask)     # Índices para el conjunto de prueba

            # Crear los conjuntos de entrenamiento y prueba usando los índices
            X_train_fold, X_test_fold = X_train[train_indices, :], X_train[test_indices, :]
            Y_train_fold, Y_test_fold = Y_train[train_indices], Y_train[test_indices]
            
            # Ajustar el modelo Lasso para el pliegue actual
            fit = glmnet(Matrix(X_train_fold), Y_train_fold, lambda=[lambda_val], alpha=1.0)
            
            # Predecir usando GLMNet.predict
            Y_pred = GLMNet.predict(fit, Matrix(X_test_fold))[:, 1]
            
            # Calcular el MSE en el conjunto de prueba
            mse = mean((Y_test_fold .- Y_pred).^2)
            all_mse[lambda_idx, fold_idx] = mse
        end
    end
    
    # Calcular el MSE promedio para cada lambda
    avg_mse = mean(all_mse, dims=2)
    
    # Encontrar el lambda óptimo (el que minimiza el MSE promedio)
    optimal_lambda_idx = argmin(avg_mse)
    optimal_lambda = all_lambdas[optimal_lambda_idx]
    
    # Ajustar el modelo final con el lambda óptimo
    fit_final = glmnet(Matrix(X_train), Y_train, lambda=[optimal_lambda], alpha=1.0)
    
    # Extraer los coeficientes directamente de `fit_final.betas`
    optimal_coef = fit_final.betas  # Coeficientes asociados al lambda óptimo
    
    # Retornar los resultados
    return Dict(
        :optimal_lambda => optimal_lambda,
        :optimal_coef => optimal_coef, 
        :all_lambdas => all_lambdas,
        :all_mse => avg_mse
    )
end
all_lambdas = λ_grid  # Usar la cuadrícula generada

# Llamar a la función de validación cruzada
resultados = cross_validation_lasso(Y_train, X_train, all_lambdas, folds)

# Imprimir el lambda óptimo y los MSEs promedio
println("Lambda óptimo: ", resultados[:optimal_lambda])
println("Coeficientes óptimos (diferentes de cero): ", resultados[:optimal_coef])
println("Todos los lambdas: ", resultados[:all_lambdas])
println("MSE por cada lambda: ", resultados[:all_mse])

In [57]:
@show resultados

Dict{Symbol, Any} with 4 entries:
  :optimal_lambda => 0.0740568
  :all_mse        => [12.5899; 11.6295; … ; 1.09069; 1.09069;;]
  :all_lambdas    => [0.001, 0.0011053, 0.00122168, 0.00135031, 0.0014925, 0.00…
  :optimal_coef   => 1327x1 CompressedPredictorMatrix:…

In [58]:
@show resultados[:optimal_coef]

1327×1 CompressedPredictorMatrix:
  0.0
  0.0
  0.0
  0.0
 -0.004181169509319653
  0.0
  0.11434850750253112
  0.14596099904094276
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

2.7. Program a function for predicting the outcome variable through model estimated with the optimal lambda. It should take as inputs \
optimal_model: A dictionary with the values outputed by the function defined for the previous point. \
X: an array of predictors. \
The output should be an array of predicted values.



In [64]:
# Función para predecir los valores usando el modelo óptimo
function predict_lasso(optimal_model::Dict, X::Matrix)
    # Extraer los coeficientes óptimos del modelo y asegurarse de que es una matriz
    optimal_coef = optimal_model[:optimal_coef]
    
    # Verificar si los coeficientes son un vector y, en ese caso, convertir a una matriz (1 columna)
    if isa(optimal_coef, Vector)
        optimal_coef = reshape(optimal_coef, :, 1)  # Convertir a matriz columna si es necesario
    end
    
    # Imprimir los coeficientes para verificar si están siendo cero
    println("Coeficientes óptimos: ", optimal_coef)
    
    # Verificar si hay un intercepto, añadiendo una columna de unos a X si es necesario
    if size(optimal_coef, 1) == size(X, 2) + 1
        X = hcat(ones(size(X, 1)), X)  # Añadir la columna de unos si hay un término de intercepto
    end
    
    # Calcular las predicciones como el producto entre los predictores X y los coeficientes
    Y_pred = X * optimal_coef

    # Retornar las predicciones
    return Y_pred
end
# Convertir X_test de DataFrame a Matrix
X_test_matrix = Matrix(X_test)

# Llamar a la función con el DataFrame convertido a matriz
Y_pred = predict_lasso(resultados, X_test_matrix)

# Mostrar las predicciones
println("Valores predichos finales: ", Y_pred)

In [65]:
@show Y_pred

515×1 Matrix{Float64}:
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
   ⋮
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN

# 3. Applying the Lasso Cross-Validation Procedure

3.8. Fit a simple OLS model with the training sample.

In [78]:
X_train_matrix = Matrix(X_train)

modelo_ols = lm(X_train_matrix, Y_train)

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
─────────────────────────────────────────────────────────────────────
               Coef.  Std. Error    t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────
x1      -0.108794            NaN  NaN       NaN        NaN        NaN
x2      14.7229              NaN  NaN       NaN        NaN        NaN
x3     -66.2514              NaN  NaN       NaN        NaN        NaN
x4     118.469               NaN  NaN       NaN        NaN        NaN
x5       0.479551            NaN  NaN       NaN        NaN        NaN
x6       0.397342            NaN  NaN       NaN        NaN        NaN
x7       1.27684             NaN  NaN       NaN        NaN        NaN
x8       1.33887             NaN  NaN       NaN        NaN        NaN
x9       0.231527            NaN  NaN       NaN        NaN        NaN
x10      0.089268

3.9. Find the optimal lambda and its related coefficients with the function programmed in 3. and the training sample. Print the lambda and the coefficients.

In [68]:
# Paso 1: Definir la cuadrícula logarítmica para los valores de lambda
lower_bound_2 = exp(-7)  # e^-7
upper_bound_2 = exp(7)   # e^7
log_spacing_2 = 0.1      # Espaciado logarítmico

# Generar la cuadrícula logarítmicamente espaciada
function log_grid_2(lower_bound_2::Float64, upper_bound_2::Float64, log_spacing_2::Float64)
    num_points_2 = Int(round((log(upper_bound_2) - log(lower_bound_2)) / log_spacing_2)) + 1
    log_grid_2 = range(log(lower_bound_2), stop=log(upper_bound_2), length=num_points_2)
    return exp.(log_grid_2)
end
# La función calcula el número de puntos en la cuadrícula, genera valores en la escala logarítmica y los convierte de nuevo a la escala original usando la función exponencial.

λ_grid_2 = log_grid_2(lower_bound_2, upper_bound_2, log_spacing_2)

# Paso 2: Generar k pliegues (5 pliegues en este caso)
k = 5  # Número de pliegues
folds = generate_k_folds(X_train, k)

# Paso 3: Ejecutar la validación cruzada con la nueva cuadrícula de lambda
resultados_ = cross_validation_lasso(Y_train, X_train, λ_grid_2, folds)

In [ ]:
resultados_[:optimal_lambda]

In [ ]:
resultados_[:optimal_coef]

3.10. Now use each language's hdm package to fit a Lasso model with the theoretical optimal lambda value. You can find the documentation of the package for each language in the following links: Julia: https://github.com/d2cml-ai/HDMjl.jl

In [ ]:
using HDMjl

In [ ]:
# Paso 4: Ajustar el modelo Lasso con el valor lambda óptimo
lasso_model = rlasso(X_train, Y_train, c = resultados_[:optimal_lambda], post = false)

summary_lasso = summary(lasso_model)  # o print_summary(lasso_model) si esa es la función correcta
summary_lasso

3.11. Report the testing MSE and R2 for the OLS model and, the cross-validation lambda Lasso model, and the hdm theoretical lambda model. Use the function programmed in 4. to compute the predictions of the cross-validation Lasso model.

In [ ]:
#modelo mco

In [ ]:
#modelo lasso model

In [ ]:
#theorical lamda model